# Week 9-1: Unsupervised Machine Learning

## Bibliotheken importeren

In [17]:
import pandas as pd
from sklearn.cluster import KMeans
import math
import matplotlib.pyplot as plt
import warnings
import sqlite3
import numpy as np
warnings.simplefilter('ignore')

## Data inlezen en kolommen selecteren

In [18]:
# go_crm_conn = sqlite3.connect("Great_Outdoors_Data_SQLite/go_crm.sqlite")
go_sales_conn = sqlite3.connect("Great_Outdoors_Data_SQLite/go_sales.sqlite")
# go_staff_conn = sqlite3.connect("Great_Outdoors_Data_SQLite/go_staff.sqlite")

# order_header = pd.read_sql("SELECT * FROM order_header", go_sales_conn)
# order_details = pd.read_sql("SELECT * FROM order_details", go_sales_conn)
# product = pd.read_sql("SELECT * FROM product", go_sales_conn)
# product_type = pd.read_sql("SELECT * FROM product_type", go_sales_conn)
# product_line = pd.read_sql("SELECT * FROM product_line", go_sales_conn)
sales_branch = pd.read_sql("SELECT * FROM sales_branch", go_sales_conn)
# country = pd.read_sql("SELECT * FROM country", go_sales_conn)

# data = pd.merge(order_header, order_details, on="ORDER_NUMBER")
# data = pd.merge(data, product, on="PRODUCT_NUMBER")
# data = pd.merge(data, product_type, on="PRODUCT_TYPE_CODE")
# data = pd.merge(data, product_line, on="PRODUCT_LINE_CODE")
# data = pd.merge(data, sales_branch, on="SALES_BRANCH_CODE")
# data = pd.merge(data, country, on="COUNTRY_CODE")

data = sales_branch
data = data.drop(["ADDRESS1", "ADDRESS2", "POSTAL_ZONE"], axis=1)
data

,SALES_BRANCH_CODE,CITY,REGION,COUNTRY_CODE,TRIAL888
0,13,Hamburg,None,2,T
1,14,München,None,2,T
2,15,Kista,None,10,T
3,17,Calgary,Alberta,4,T
4,18,Toronto,Ontario,4,T
5,19,Boston,Massachusetts,3,T
6,20,Seattle,Washington,3,T
7,21,Los Angeles,California,3,T
8,22,Miami,Florida,3,T
9,23,Lyon,None,1,T


In [19]:
# data = data.drop_duplicates(["SALES_BRANCH_CODE"])

# data = data.loc[:, ["PRODUCT_LINE_CODE", "COUNTRY", "CITY", "SALES_BRANCH_CODE"]]
data

,SALES_BRANCH_CODE,CITY,REGION,COUNTRY_CODE,TRIAL888
0,13,Hamburg,None,2,T
1,14,München,None,2,T
2,15,Kista,None,10,T
3,17,Calgary,Alberta,4,T
4,18,Toronto,Ontario,4,T
5,19,Boston,Massachusetts,3,T
6,20,Seattle,Washington,3,T
7,21,Los Angeles,California,3,T
8,22,Miami,Florida,3,T
9,23,Lyon,None,1,T


## One-hot encoding van onafhankelijke niet-numerieke variabelen

In [20]:
dummies = pd.get_dummies(data.loc[:,["CITY", "REGION"]])

dummies

,CITY_Amsterdam,CITY_Bilbao,CITY_Birmingham,CITY_Boston,CITY_Calgary,CITY_Distrito Federal,CITY_Hamburg,CITY_Heverlee,CITY_Kista,CITY_Kuopio,...,REGION_Noord-Holland,REGION_Ontario,REGION_Osaka,REGION_SP,REGION_Taipei,REGION_Tokyo,REGION_VIC,REGION_Vizcaya,REGION_Washington,REGION_West Midlands
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
df = pd.concat([data, dummies], axis = 1)
# df.drop(["COUNTRY", "CITY"], axis=1, inplace=True)
df

,SALES_BRANCH_CODE,CITY,REGION,COUNTRY_CODE,TRIAL888,CITY_Amsterdam,CITY_Bilbao,CITY_Birmingham,CITY_Boston,CITY_Calgary,...,REGION_Noord-Holland,REGION_Ontario,REGION_Osaka,REGION_SP,REGION_Taipei,REGION_Tokyo,REGION_VIC,REGION_Vizcaya,REGION_Washington,REGION_West Midlands
0,13,Hamburg,None,2,T,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,14,München,None,2,T,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,15,Kista,None,10,T,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,17,Calgary,Alberta,4,T,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,18,Toronto,Ontario,4,T,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5,19,Boston,Massachusetts,3,T,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,20,Seattle,Washington,3,T,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7,21,Los Angeles,California,3,T,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,22,Miami,Florida,3,T,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,23,Lyon,None,1,T,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Clusteringmodel bouwen met 2 dimensies

In [22]:
df_2d = dummies.loc[:, :]
# df_2d = pd.get_dummies(df_2d, columns=[])
df_2d

,CITY_Amsterdam,CITY_Bilbao,CITY_Birmingham,CITY_Boston,CITY_Calgary,CITY_Distrito Federal,CITY_Hamburg,CITY_Heverlee,CITY_Kista,CITY_Kuopio,...,REGION_Noord-Holland,REGION_Ontario,REGION_Osaka,REGION_SP,REGION_Taipei,REGION_Tokyo,REGION_VIC,REGION_Vizcaya,REGION_Washington,REGION_West Midlands
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
5,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
# TODO: parameters aanpassen om het beste resultaat te vinden
kmeans = KMeans(n_clusters = 2, random_state = 42)      # Aanpassing van het aantal clusters maakt een verschil, maar krijg ik niet beter dan het nu is
kmeans.fit_predict(df_2d)

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [24]:
kmeans_centra = pd.DataFrame(kmeans.cluster_centers_)
kmeans_centra

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00,0.000000
1,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,...,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,1.387779e-17,0.037037


In [25]:
for i in range(len(kmeans_centra.columns)):
    kmeans_centra = kmeans_centra.rename(columns = {i : f'{df_2d.columns[i]}'})
    
kmeans_centra

,CITY_Amsterdam,CITY_Bilbao,CITY_Birmingham,CITY_Boston,CITY_Calgary,CITY_Distrito Federal,CITY_Hamburg,CITY_Heverlee,CITY_Kista,CITY_Kuopio,...,REGION_Noord-Holland,REGION_Ontario,REGION_Osaka,REGION_SP,REGION_Taipei,REGION_Tokyo,REGION_VIC,REGION_Vizcaya,REGION_Washington,REGION_West Midlands
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+00,0.000000
1,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,...,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,0.037037,1.387779e-17,0.037037


In [26]:
for src_index, _ in df_2d.iterrows():
    euclidian_distances = dict()
    print(f"Afstand van bronindex {src_index} tot...")

    for centrumindex, _ in kmeans_centra.iterrows():
        print(f"\tCentrumindex {centrumindex}:")
        euclidian_sum = 0

        for column_name in kmeans_centra.columns:
            current_difference = df_2d.at[src_index, column_name] - kmeans_centra.at[centrumindex, column_name]
            print(f'\t\t{df_2d.at[src_index, column_name]} - {kmeans_centra.at[centrumindex, column_name]} = {current_difference}')
            euclidian_sum += current_difference ** 2
        
        print(f'\tTotale euclidische som: {euclidian_sum}')
        euclidian_distance = math.sqrt(euclidian_sum)
        print(f'\tEuclidische afstand: {euclidian_distance}')
        euclidian_distances[centrumindex] = euclidian_distance
        print('------------------------------------------------')
    
    print(euclidian_distances)
    centrum_number = min(euclidian_distances, key = euclidian_distances.get)
    print(centrum_number)
    df_2d.at[src_index, 'Centrum'] = centrum_number
    print("================================================")

df_2d

Afstand van bronindex 0 tot...
	Centrumindex 0:
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		1 - 0.0 = 1.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 1.0 = -1.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 0.0 = 0.0
		0 - 1.0 = -1.0
		0 - 0.0 = 0.0
	Totale euclidische som: 3.0
	Euclidische afstand: 1.7320508075688772
------------------------------------------------
	Centrumindex 1:
		0 - 0.03703703703703705 = -0.03703703703703705
		0 - 0.03703703703703705 = -0.03703703703703705
		0 - 0.03703703703703704 = 

,CITY_Amsterdam,CITY_Bilbao,CITY_Birmingham,CITY_Boston,CITY_Calgary,CITY_Distrito Federal,CITY_Hamburg,CITY_Heverlee,CITY_Kista,CITY_Kuopio,...,REGION_Ontario,REGION_Osaka,REGION_SP,REGION_Taipei,REGION_Tokyo,REGION_VIC,REGION_Vizcaya,REGION_Washington,REGION_West Midlands,Centrum
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1.0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
4,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1.0
5,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0.0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1.0


In [27]:
# plt.scatter(df_2d['SALES_BRANCH_CODE'], df_2d['PRODUCT_LINE_CODE'], color = 'k')
# plt.show()

In [28]:
df_2d.groupby('Centrum', as_index = False)['Centrum'].count()

,Centrum
0,1
1,27


## Clusteringmodel bouwen met meer dan 2 dimensies (alle kolommen uit de dataset)

### Centra berekenen

In [29]:
# TODO: finetunen van het aantal clusters
kmeans = KMeans(n_clusters = 2, random_state = 42)      # Aantal clusters lijkt geen verschil te maken... Seed ook niet...
prediction_results = kmeans.fit_predict(df_2d)
prediction_results

array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1])

In [30]:
df['Centrum'] = prediction_results
df

,SALES_BRANCH_CODE,CITY,REGION,COUNTRY_CODE,TRIAL888,CITY_Amsterdam,CITY_Bilbao,CITY_Birmingham,CITY_Boston,CITY_Calgary,...,REGION_Ontario,REGION_Osaka,REGION_SP,REGION_Taipei,REGION_Tokyo,REGION_VIC,REGION_Vizcaya,REGION_Washington,REGION_West Midlands,Centrum
0,13,Hamburg,None,2,T,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,14,München,None,2,T,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,15,Kista,None,10,T,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,17,Calgary,Alberta,4,T,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,18,Toronto,Ontario,4,T,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
5,19,Boston,Massachusetts,3,T,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
6,20,Seattle,Washington,3,T,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
7,21,Los Angeles,California,3,T,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8,22,Miami,Florida,3,T,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
9,23,Lyon,None,1,T,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### De juiste centra toewijzen aan rijen uit de dataset

In [31]:
df.groupby('Centrum', as_index = False)['Centrum'].count()

,Centrum
0,1
1,27


## Evalueren

In [32]:
intercluster_distance = 0
intracluster_distance = 0

common_columns = df_2d.columns.intersection(kmeans_centra.columns)

for centrumindex, _ in kmeans_centra[common_columns].iterrows():
    for src_index, _ in df_2d[common_columns].iterrows():
        if df_2d.at[src_index, 'Centrum'] == centrumindex:
            diff = df_2d.loc[src_index, common_columns] - kmeans_centra.loc[centrumindex, common_columns]
            distance = np.linalg.norm(diff[pd.to_numeric(diff, errors='coerce').notnull()])
            intracluster_distance += distance
        else:
            diff = df_2d.loc[src_index, common_columns] - kmeans_centra.loc[centrumindex, common_columns]
            distance = np.linalg.norm(diff[pd.to_numeric(diff, errors='coerce').notnull()])
            intercluster_distance += distance

print(f"Intercluster distance (more is better): {intercluster_distance}")
print(f"Intracluster distance (less is better): {intracluster_distance}")

Intercluster distance (more is better): 51.95062054895879
Intracluster distance (less is better): 32.22229132362789
